In [39]:
#import modules
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn.cross_validation import cross_val_score

In [8]:
data_train = pd.read_csv('data/Clean.csv')
data_test = pd.read_csv('data/CleanTest.csv')

In [3]:
data_train.head()

,Unnamed: 0,AnimalID,Name,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color,Day,Month,Year,Hours,Minutes
0,1,A671945,Hambone,Return_to_owner,NaN,Dog,Neutered Male,1 year,Shetland Sheepdog Mix,Brown/White,12,2,2014,18,22
1,2,A656520,Emily,Euthanasia,Suffering,Cat,Spayed Female,1 year,Domestic Shorthair Mix,Cream Tabby,13,10,2013,12,44
2,3,A686464,Pearce,Adoption,Foster,Dog,Neutered Male,2 years,Pit Bull Mix,Blue/White,31,1,2015,12,28
3,4,A683430,NaN,Transfer,Partner,Cat,Intact Male,3 weeks,Domestic Shorthair Mix,Blue Cream,11,7,2014,19,9
4,5,A667013,NaN,Transfer,Partner,Dog,Neutered Male,2 years,Lhasa Apso/Miniature Poodle,Tan,15,11,2013,12,52


In [4]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26710 entries, 0 to 26709
Data columns (total 15 columns):
Unnamed: 0        26710 non-null int64
AnimalID          26710 non-null object
Name              19037 non-null object
OutcomeType       26710 non-null object
OutcomeSubtype    13099 non-null object
AnimalType        26710 non-null object
SexuponOutcome    26710 non-null object
AgeuponOutcome    26710 non-null object
Breed             26710 non-null object
Color             26710 non-null object
Day               26710 non-null int64
Month             26710 non-null int64
Year              26710 non-null int64
Hours             26710 non-null int64
Minutes           26710 non-null int64
dtypes: int64(6), object(9)
memory usage: 3.3+ MB


In [7]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11456 entries, 0 to 11455
Data columns (total 13 columns):
Unnamed: 0        11456 non-null int64
ID                11456 non-null int64
Name              8231 non-null object
AnimalType        11456 non-null object
SexuponOutcome    11456 non-null object
AgeuponOutcome    11450 non-null object
Breed             11456 non-null object
Color             11456 non-null object
Day               11456 non-null int64
Month             11456 non-null int64
Year              11456 non-null int64
Hours             11456 non-null int64
Minutes           11456 non-null int64
dtypes: int64(7), object(6)
memory usage: 1.2+ MB


In [98]:
train = data_train.drop(['AnimalID', 'Name', 'OutcomeSubtype', 'OutcomeType', 'Unnamed: 0'], axis = 1)
test = data_test.drop(['ID', 'Name', 'Unnamed: 0'], axis = 1)

In [99]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26710 entries, 0 to 26709
Data columns (total 10 columns):
AnimalType        26710 non-null object
SexuponOutcome    26710 non-null object
AgeuponOutcome    26710 non-null object
Breed             26710 non-null object
Color             26710 non-null object
Day               26710 non-null int64
Month             26710 non-null int64
Year              26710 non-null int64
Hours             26710 non-null int64
Minutes           26710 non-null int64
dtypes: int64(5), object(5)
memory usage: 2.2+ MB


In [100]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11456 entries, 0 to 11455
Data columns (total 10 columns):
AnimalType        11456 non-null object
SexuponOutcome    11456 non-null object
AgeuponOutcome    11450 non-null object
Breed             11456 non-null object
Color             11456 non-null object
Day               11456 non-null int64
Month             11456 non-null int64
Year              11456 non-null int64
Hours             11456 non-null int64
Minutes           11456 non-null int64
dtypes: int64(5), object(5)
memory usage: 984.5+ KB


In [101]:
lb = LabelEncoder()
categorical_columns = train.columns[train.dtypes == 'object']
for var in categorical_columns:
    full_data = pd.concat((train[var],test[var]),axis=0).astype('str')
    lb.fit(full_data )
    train.loc[:, var] = lb.transform(train[var].astype('str'))
    test.loc[:, var] = lb.transform(test[var].astype('str'))

In [102]:
train.head()

,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color,Day,Month,Year,Hours,Minutes
0,1,2,5,1482,146,12,2,2014,18,22
1,0,3,5,775,184,13,10,2013,12,44
2,1,2,21,1293,97,31,1,2015,12,28
3,0,1,26,775,47,11,7,2014,19,9
4,1,2,21,1101,311,15,11,2013,12,52


In [103]:
test.fillna(train.mean(), inplace=True);

In [104]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11456 entries, 0 to 11455
Data columns (total 10 columns):
AnimalType        11456 non-null int64
SexuponOutcome    11456 non-null int64
AgeuponOutcome    11456 non-null int64
Breed             11456 non-null int64
Color             11456 non-null int64
Day               11456 non-null int64
Month             11456 non-null int64
Year              11456 non-null int64
Hours             11456 non-null int64
Minutes           11456 non-null int64
dtypes: int64(10)
memory usage: 984.5 KB


In [105]:
data_x = train.values
data_y = data_train.OutcomeType.values

data_x_test = test.values

In [106]:
clf1 = RandomForestClassifier(n_jobs=3, n_estimators=250, oob_score=True)

In [107]:
clf2 = GradientBoostingClassifier(n_estimators=250)

In [25]:
# rmf100
cross_val_score(clf1, data_x, data_y, cv = 3, scoring = 'log_loss')

array([-0.95459233, -0.98648117, -0.91576981])

In [87]:
# rmf250_oob
cross_val_score(clf1, data_x, data_y, cv = 3, scoring = 'log_loss')

array([-0.93166129, -0.95201874, -0.91833644])

In [116]:
# xgb250
cross_val_score(clf2, data_x, data_y, cv = 3, scoring = 'log_loss')

array([-0.83455612, -0.81797369, -0.81823957])

In [43]:
# xgb100
cross_val_score(clf2, data_x, data_y, cv = 3, scoring = 'log_loss')

array([-0.84642536, -0.83332468, -0.83589798])

In [108]:
clf2.fit(data_x, data_y);
pred = clf2.predict_proba(data_x_test)

In [110]:
sub = pd.DataFrame(pred, columns=clf2.classes_)

In [111]:
sub['ID'] = data_test.ID

In [112]:
sub.to_csv('sub/sub01.csv', index=False)